In [100]:
import pandas as pd
from gurobipy import *
from pyomo.environ import *

In [101]:
xls = pd.ExcelFile('Proyecto_1_TransporteMaritimo_Dataset1.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])

Se transforman los datos de los dataframes a diccionarios para el procesamiento de datos

In [102]:
barcos_sheet = df1.to_dict("index")
compatibilidad_sheet = df2.to_dict()
cargos_sheet = df3.to_dict("index")
costos_transporte_sheet = df4.to_dict("index")
costos_transporte_dict = {(costos_transporte_sheet[i]['ID_Barco'],costos_transporte_sheet[i]['Puerto_Origen'],costos_transporte_sheet[i]['Puerto_Destino']) : {'Tiempo_Viaje_hrs':costos_transporte_sheet[i]['Tiempo_Viaje_hrs'], 'Costo_Viaje_libras':costos_transporte_sheet[i]['Costo_Viaje_libras']} for i in costos_transporte_sheet.keys()}
costos_tiempo_sheet = df5.to_dict("index")
costos_tiempo_dict = {(costos_tiempo_sheet[i]['ID_Barco'],costos_tiempo_sheet[i]['ID_Cargo']) : {'Tiempo_Origen_hrs':costos_tiempo_sheet[i]['Tiempo_Origen_hrs'], 'Costos_Origen_hrs':costos_tiempo_sheet[i]['Costos_Origen_hrs'], 'Tiempo_Destino_hrs':costos_tiempo_sheet[i]['Tiempo_Destino_hrs'], 'Costos_Destino_hrs':costos_tiempo_sheet[i]['Costos_Destino_hrs']} for i in costos_tiempo_sheet.keys()}

In [103]:
compatibilidad_sheet

{'ID_Barco': {0: 1, 1: 2, 2: 3, 3: 4},
 'ID_Cargo_Compatible': {0: 1, 1: 1, 2: 1, 3: 1},
 'Unnamed: 2': {0: 2, 1: 3, 2: 3, 3: 5},
 'Unnamed: 3': {0: 3, 1: 4, 2: 5, 3: 7},
 'Unnamed: 4': {0: 4, 1: 5, 2: 7, 3: 9},
 'Unnamed: 5': {0: 5, 1: 7, 2: 9, 3: 10},
 'Unnamed: 6': {0: 6, 1: 9, 2: 10, 3: 12},
 'Unnamed: 7': {0: 7, 1: 10, 2: 12, 3: 14},
 'Unnamed: 8': {0: 8.0, 1: 11.0, 2: nan, 3: nan},
 'Unnamed: 9': {0: 10.0, 1: 12.0, 2: nan, 3: nan},
 'Unnamed: 10': {0: 11.0, 1: 14.0, 2: nan, 3: nan},
 'Unnamed: 11': {0: 12.0, 1: 15.0, 2: nan, 3: nan},
 'Unnamed: 12': {0: 13.0, 1: nan, 2: nan, 3: nan},
 'Unnamed: 13': {0: 15.0, 1: nan, 2: nan, 3: nan}}

In [104]:
n_cargos = len(cargos_sheet)
n_cargos

15

 La manera en que funcionan los nodos es: los primeros $n$ ($n =$ número de cargos) son los nodos de carga, los siguientes $n$ son los nodos de descarga, los siguientes $v$ ($v =$ número de barcos) son los nodos de origen, y los último v son los nodos de destino.

In [105]:
V = [1+i for i in barcos_sheet.keys()] ## conjunto barcos
n_barcos = len(V)
print(V)

[1, 2, 3, 4]


In [106]:
Kv= {} # conjunto capacidades de barcos
for i in barcos_sheet.keys():
    Kv[barcos_sheet[i]['ID_Barco']] =  barcos_sheet[i]['Capacidad']
Kv

{1: 16500, 2: 5800, 3: 3570, 4: 3570}

In [107]:
# N^P
nodos_carga= [cargos_sheet[i]['ID_Cargo'] for i in cargos_sheet.keys()]
print(nodos_carga)

#N^D
nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga
print(nodos_descarga)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [108]:
nodos = {}
for i in range(n_cargos):
    nodos[cargos_sheet[i]['ID_Cargo']] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Origen'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Carga'],
                                          'RT': cargos_sheet[i]['RT_Carga']}
    
    nodos[cargos_sheet[i]['ID_Cargo']+n_cargos] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Destino'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Descarga'],
                                          'RT': cargos_sheet[i]['RT_Descarga']}
o = {}
d = {}
for v in V:
    o[v] = n_cargos*2+v
    nodos[n_cargos*2+v] = {'Costo_SPOT': 0,
                           'ID_Cargo': 0,
                           'ID_Puerto': barcos_sheet[v-1]['ID_Puerto '],
                           'LT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'RT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'Tamano': 0}
    d[v] = n_cargos*2+n_barcos+v
    nodos[n_cargos*2+n_barcos+v] = {'Costo_SPOT': 0,
                                    'ID_Cargo': 0,
                                    'ID_Puerto': 0,
                                    'LT': 100000,
                                    'RT': 100000,
                                    'Tamano': 0}
print(o) # nodos de origen
print(d) # nodos de destino
print(nodos) # todos los nodos con su información

{1: 31, 2: 32, 3: 33, 4: 34}
{1: 35, 2: 36, 3: 37, 4: 38}
{1: {'ID_Cargo': 1, 'ID_Puerto': 25, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 455}, 16: {'ID_Cargo': 1, 'ID_Puerto': 13, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 777}, 2: {'ID_Cargo': 2, 'ID_Puerto': 9, 'Tamano': 6919, 'Costo_SPOT': 536901, 'LT': 149, 'RT': 221}, 17: {'ID_Cargo': 2, 'ID_Puerto': 6, 'Tamano': 6919, 'Costo_SPOT': 536901, 'LT': 149, 'RT': 588}, 3: {'ID_Cargo': 3, 'ID_Puerto': 7, 'Tamano': 3395, 'Costo_SPOT': 474427, 'LT': 5, 'RT': 77}, 18: {'ID_Cargo': 3, 'ID_Puerto': 4, 'Tamano': 3395, 'Costo_SPOT': 474427, 'LT': 5, 'RT': 489}, 4: {'ID_Cargo': 4, 'ID_Puerto': 6, 'Tamano': 5688, 'Costo_SPOT': 458905, 'LT': 244, 'RT': 316}, 19: {'ID_Cargo': 4, 'ID_Puerto': 1, 'Tamano': 5688, 'Costo_SPOT': 458905, 'LT': 244, 'RT': 624}, 5: {'ID_Cargo': 5, 'ID_Puerto': 38, 'Tamano': 3419, 'Costo_SPOT': 644852, 'LT': 338, 'RT': 410}, 20: {'ID_Cargo': 5, 'ID_Puerto': 33, 'Tamano': 3419, 'Costo_SPOT': 644852, 

In [109]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
    Nv[i] = []
    for j in nodos.keys():
        if nodos[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
            Nv[i].append(j)
Nv

{1: [1,
  16,
  2,
  17,
  3,
  18,
  4,
  19,
  5,
  20,
  6,
  21,
  7,
  22,
  8,
  23,
  10,
  25,
  11,
  26,
  12,
  27,
  13,
  28,
  15,
  30,
  31,
  35],
 2: [1,
  16,
  3,
  18,
  4,
  19,
  5,
  20,
  7,
  22,
  9,
  24,
  10,
  25,
  11,
  26,
  12,
  27,
  14,
  29,
  15,
  30,
  32,
  36],
 3: [1, 16, 3, 18, 5, 20, 7, 22, 9, 24, 10, 25, 12, 27, 33, 37],
 4: [1, 16, 5, 20, 7, 22, 9, 24, 10, 25, 12, 27, 14, 29, 34, 38]}

In [110]:
"""import itertools
Av = {}
for v in V:
    Av[v] = list(itertools.permutations(Nv[v],2))
"""
Av={}
indices = []
for v in V:
    Av[v] = [] 
    for i in Nv[v]:
        for j in Nv[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)
print(Av[1])

[(1, 1), (1, 16), (1, 2), (1, 17), (1, 3), (1, 18), (1, 4), (1, 19), (1, 5), (1, 20), (1, 6), (1, 21), (1, 7), (1, 22), (1, 8), (1, 23), (1, 10), (1, 25), (1, 11), (1, 26), (1, 12), (1, 27), (1, 13), (1, 28), (1, 15), (1, 30), (1, 31), (1, 35), (16, 1), (16, 16), (16, 2), (16, 17), (16, 3), (16, 18), (16, 4), (16, 19), (16, 5), (16, 20), (16, 6), (16, 21), (16, 7), (16, 22), (16, 8), (16, 23), (16, 10), (16, 25), (16, 11), (16, 26), (16, 12), (16, 27), (16, 13), (16, 28), (16, 15), (16, 30), (16, 31), (16, 35), (2, 1), (2, 16), (2, 2), (2, 17), (2, 3), (2, 18), (2, 4), (2, 19), (2, 5), (2, 20), (2, 6), (2, 21), (2, 7), (2, 22), (2, 8), (2, 23), (2, 10), (2, 25), (2, 11), (2, 26), (2, 12), (2, 27), (2, 13), (2, 28), (2, 15), (2, 30), (2, 31), (2, 35), (17, 1), (17, 16), (17, 2), (17, 17), (17, 3), (17, 18), (17, 4), (17, 19), (17, 5), (17, 20), (17, 6), (17, 21), (17, 7), (17, 22), (17, 8), (17, 23), (17, 10), (17, 25), (17, 11), (17, 26), (17, 12), (17, 27), (17, 13), (17, 28), (17, 15

In [111]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]

print(NP_v)
print(ND_v)

{1: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15], 2: [1, 3, 4, 5, 7, 9, 10, 11, 12, 14, 15], 3: [1, 3, 5, 7, 9, 10, 12], 4: [1, 5, 7, 9, 10, 12, 14]}
{1: [16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30], 2: [16, 18, 19, 20, 22, 24, 25, 26, 27, 29, 30], 3: [16, 18, 20, 22, 24, 25, 27], 4: [16, 20, 22, 24, 25, 27, 29]}


In [112]:
LT_i = {i:nodos[i]["LT"] for i in nodos.keys()} #lower bounds ventana de tiempo por nodo
RT_i = {i:nodos[i]["RT"] for i in nodos.keys()} #upper bounds ventana de tiempo por nodo

print(LT_i)
print(RT_i)

{1: 383, 16: 383, 2: 149, 17: 149, 3: 5, 18: 5, 4: 244, 19: 244, 5: 338, 20: 338, 6: 127, 21: 127, 7: 621, 22: 621, 8: 238, 23: 238, 9: 39, 24: 39, 10: 698, 25: 698, 11: 94, 26: 94, 12: 572, 27: 572, 13: 842, 28: 842, 14: 100, 29: 100, 15: 325, 30: 325, 31: 69, 35: 100000, 32: 0, 36: 100000, 33: 0, 37: 100000, 34: 0, 38: 100000}
{1: 455, 16: 777, 2: 221, 17: 588, 3: 77, 18: 489, 4: 316, 19: 624, 5: 410, 20: 871, 6: 199, 21: 657, 7: 693, 22: 982, 8: 310, 23: 967, 9: 111, 24: 501, 10: 770, 25: 1198, 11: 166, 26: 497, 12: 644, 27: 1093, 13: 914, 28: 1388, 14: 172, 29: 800, 15: 397, 30: 746, 31: 69, 35: 100000, 32: 0, 36: 100000, 33: 0, 37: 100000, 34: 0, 38: 100000}


In [113]:
T_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in nodos.keys():
    T_ijv[i] = {} 
    for j in nodos.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                T_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Tiempo_Viaje_hrs']
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Origen_hrs']
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Destino_hrs']
                
                # se les asigna tiempo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el tiempo entre el puerto de i y j

In [114]:
C_ijv={} # costo de entre i y j (costo de viaje + costo de operación en i)
for i in nodos.keys():
    C_ijv[i] = {} 
    for j in nodos.keys():
        C_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                C_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Costo_Viaje_libras']
                if i==11 and j==2 and i==1:
                    print(C_ijv[i][j][v])
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Origen_hrs']

                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Destino_hrs']

In [115]:
Qj={i:nodos[i]["Tamano"] for i in nodos.keys()}
print("Qj: ",Qj)

print("\n")

Cs = {cargos_sheet[i]['ID_Cargo']:cargos_sheet[i]['Costo_SPOT'] for i in cargos_sheet.keys()}
print("Cs", Cs)

Qj:  {1: 2368, 16: 2368, 2: 6919, 17: 6919, 3: 3395, 18: 3395, 4: 5688, 19: 5688, 5: 3419, 20: 3419, 6: 7277, 21: 7277, 7: 2733, 22: 2733, 8: 6074, 23: 6074, 9: 2714, 24: 2714, 10: 1497, 25: 1497, 11: 4813, 26: 4813, 12: 2370, 27: 2370, 13: 7064, 28: 7064, 14: 2308, 29: 2308, 15: 4730, 30: 4730, 31: 0, 35: 0, 32: 0, 36: 0, 33: 0, 37: 0, 34: 0, 38: 0}


Cs {1: 354894, 2: 536901, 3: 474427, 4: 458905, 5: 644852, 6: 733327, 7: 276767, 8: 1037825, 9: 326529, 10: 420165, 11: 461936, 12: 547778, 13: 893561, 14: 844864, 15: 306945}


In [116]:
nodos_index = [i for i in nodos.keys()]

In [117]:
1<2

True

In [118]:
def obj_rule(model):
    return sum(C_ijv[i][j][v]*model.x[i,j,v]  for v in V for (i,j) in Av[v])+sum(Cs[i]*model.y[i] for i in nodos_carga)

def const_1(model, i):
    return sum(model.x[i,j,v]  for v in V for j in Nv[v]) + model.y[i] == 1

def const_2(model, v):
    return sum(model.x[o[v],j,v] for j in Nv[v]) == 1

def const_3(model, i, v):
    if i in Nv[v]:
        if i!=o[v] and i!=d[v]:
            #return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[j,i,v] for j in Nv[v] if j!=i)==0
            return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[j,i,v] for j in Nv[v])==0
    return Constraint.Skip

def const_4(model, v):
    return sum(model.x[j,d[v],v] for j in Nv[v]) == 1
def const_5(model, i, j, v):
    if j in NP_v[v]:
        if (i,j) in Av[v]:
            return model.l[i,v]+Qj[j]-model.l[j,v]<=Kv[v]*(1-model.x[i,j,v])
    return Constraint.Skip

def const_6(model, i, j, v):
    if j in NP_v[v]:
        if (i,n_cargos+j) in Av[v]:
            return model.l[i,v]-Qj[j]-model.l[n_cargos+j,v]<=Kv[v]*(1-model.x[i,n_cargos+j,v])
    return Constraint.Skip

def const_7(model, i, v):
    if i in NP_v[v]:
        return (0, model.l[i,v], Kv[v])
    return Constraint.Skip

def const_8(model, i, j, v):
    if (i,j) in Av[v]:
        return model.t[i,v]+T_ijv[i][j][v]-model.t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-model.x[i,j,v])
    return Constraint.Skip

def const_9(model, i, v):
    if i in NP_v[v]:
        #return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[n_cargos+i,j,v] for j in Nv[v] if j!=i+n_cargos)==0
        return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[n_cargos+i,j,v] for j in Nv[v])==0
    return Constraint.Skip

def const_10(model, i, v):
    if i in NP_v[v]:
        return model.t[i,v]+T_ijv[i][n_cargos+i][v]-model.t[n_cargos+i,v]<=0
    return Constraint.Skip
    
def const_11(model, i, v):
    if i in Nv[v]:
        return (LT_i[i], model.t[i,v], RT_i[i])
    return Constraint.Skip
"""

def const_7_ub(model, i, v):
    if i in NP_v[v]:
        return model.l[i,v]<=Kv[v]
    return Constraint.Skip

def const_7_lb(model, i, v):
    if i in NP_v[v]:
        return 0<=model.l[i,v]
    return Constraint.Skip
    
def const_11_ub(model, i, v):
    if i in Nv[v]:
        return model.t[i,v]<=RT_i[i]
    return Constraint.Skip

def const_11_lb(model, i, v):
    if i in Nv[v]:
        return LT_i[i]<=model.t[i,v]
    return Constraint.Skip

"""

'\n\ndef const_7_ub(model, i, v):\n    if i in NP_v[v]:\n        return model.l[i,v]<=Kv[v]\n    return Constraint.Skip\n\ndef const_7_lb(model, i, v):\n    if i in NP_v[v]:\n        return 0<=model.l[i,v]\n    return Constraint.Skip\n    \ndef const_11_ub(model, i, v):\n    if i in Nv[v]:\n        return model.t[i,v]<=RT_i[i]\n    return Constraint.Skip\n\ndef const_11_lb(model, i, v):\n    if i in Nv[v]:\n        return LT_i[i]<=model.t[i,v]\n    return Constraint.Skip\n\n'

In [119]:
model_1 = ConcreteModel()

In [120]:
model_1.x = Var(nodos_index, nodos_index, V, within=Binary)
model_1.y = Var(nodos_carga, within=Binary)
model_1.t = Var(nodos_index,V)
model_1.l = Var(nodos_index,V, within=NonNegativeReals)

In [121]:
Kv[1]

16500

In [122]:
model_1.obj = Objective(rule=obj_rule)
model_1.const_1 = Constraint(nodos_carga, rule=const_1)
model_1.const_2 = Constraint(V, rule=const_2)
model_1.const_3 = Constraint(nodos_index, V, rule=const_3)
model_1.const_4 = Constraint(V, rule=const_4)
model_1.const_5 = Constraint(nodos_index, nodos_index, V, rule=const_5)
model_1.const_6 = Constraint(nodos_index, nodos_index, V, rule=const_6)
model_1.const_7 = Constraint(nodos_index, V, rule=const_7)
model_1.const_8 = Constraint(nodos_index, nodos_index, V, rule=const_8)
model_1.const_9 = Constraint(nodos_index, V, rule=const_9)
model_1.const_10 = Constraint(nodos_index, V, rule=const_10)
model_1.const_11 = Constraint(nodos_index, V, rule=const_11)

"""
model_1.const_7_ub = Constraint(nodos_index, V, rule=const_7_ub)
model_1.const_7_lb = Constraint(nodos_index, V, rule=const_7_lb)
model_1.const_11_ub = Constraint(nodos_index, V, rule=const_11_ub)
model_1.const_11_lb = Constraint(nodos_index, V, rule=const_11_lb)
"""

'\nmodel_1.const_7_ub = Constraint(nodos_index, V, rule=const_7_ub)\nmodel_1.const_7_lb = Constraint(nodos_index, V, rule=const_7_lb)\nmodel_1.const_11_ub = Constraint(nodos_index, V, rule=const_11_ub)\nmodel_1.const_11_lb = Constraint(nodos_index, V, rule=const_11_lb)\n'

In [123]:
solver_1 = SolverFactory('gurobi')
solver_1.solve(model_1)['Solver'][0]['Message']

'Model was solved to optimality (subject to tolerances), and an optimal solution is available.'

In [124]:
model_1.obj()

363924.0

In [125]:
model_1.x.pprint()

x : Size=5776, Index=x_index
    Key         : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
      (1, 1, 2) :     0 :   0.0 :     1 : False : False : Binary
      (1, 1, 3) :     0 :   0.0 :     1 : False : False : Binary
      (1, 1, 4) :     0 :   0.0 :     1 : False : False : Binary
      (1, 2, 1) :     0 :   0.0 :     1 : False : False : Binary
      (1, 2, 2) :     0 :  None :     1 : False :  True : Binary
      (1, 2, 3) :     0 :  None :     1 : False :  True : Binary
      (1, 2, 4) :     0 :  None :     1 : False :  True : Binary
      (1, 3, 1) :     0 :   0.0 :     1 : False : False : Binary
      (1, 3, 2) :     0 :   0.0 :     1 : False : False : Binary
      (1, 3, 3) :     0 :   0.0 :     1 : False : False : Binary
      (1, 3, 4) :     0 :  None :     1 : False :  True : Binary
      (1, 4, 1) :     0 :   0.0 :     1 : False : False : Binary
      (1, 4, 2) :     0 :   0.0 :     1 : False : False : Bin

    (11, 19, 3) :     0 :  None :     1 : False :  True : Binary
    (11, 19, 4) :     0 :  None :     1 : False :  True : Binary
    (11, 20, 1) :     0 :  -0.0 :     1 : False : False : Binary
    (11, 20, 2) :     0 :   0.0 :     1 : False : False : Binary
    (11, 20, 3) :     0 :   0.0 :     1 : False : False : Binary
    (11, 20, 4) :     0 :   0.0 :     1 : False : False : Binary
    (11, 21, 1) :     0 :   0.0 :     1 : False : False : Binary
    (11, 21, 2) :     0 :  None :     1 : False :  True : Binary
    (11, 21, 3) :     0 :  None :     1 : False :  True : Binary
    (11, 21, 4) :     0 :  None :     1 : False :  True : Binary
    (11, 22, 1) :     0 :  -0.0 :     1 : False : False : Binary
    (11, 22, 2) :     0 :   0.0 :     1 : False : False : Binary
    (11, 22, 3) :     0 :   0.0 :     1 : False : False : Binary
    (11, 22, 4) :     0 :   0.0 :     1 : False : False : Binary
    (11, 23, 1) :     0 :  -0.0 :     1 : False : False : Binary
    (11, 23, 2) :     0 :

    (27, 36, 2) :     0 :   1.0 :     1 : False : False : Binary
    (27, 36, 3) :     0 :  None :     1 : False :  True : Binary
    (27, 36, 4) :     0 :  None :     1 : False :  True : Binary
    (27, 37, 1) :     0 :  None :     1 : False :  True : Binary
    (27, 37, 2) :     0 :  None :     1 : False :  True : Binary
    (27, 37, 3) :     0 :  -0.0 :     1 : False : False : Binary
    (27, 37, 4) :     0 :  None :     1 : False :  True : Binary
    (27, 38, 1) :     0 :  None :     1 : False :  True : Binary
    (27, 38, 2) :     0 :  None :     1 : False :  True : Binary
    (27, 38, 3) :     0 :  None :     1 : False :  True : Binary
    (27, 38, 4) :     0 :  -0.0 :     1 : False : False : Binary
     (28, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
     (28, 1, 2) :     0 :  None :     1 : False :  True : Binary
     (28, 1, 3) :     0 :  None :     1 : False :  True : Binary
     (28, 1, 4) :     0 :  None :     1 : False :  True : Binary
     (28, 2, 1) :     0 :

In [126]:
for i in model_1.x:
    if model_1.x[i].value == 1:
        print(i)

(1, 12, 2)
(16, 27, 2)
(2, 38, 4)
(3, 38, 4)
(4, 38, 4)
(5, 20, 4)
(20, 38, 4)
(6, 38, 4)
(7, 22, 1)
(22, 35, 1)
(8, 38, 4)
(9, 35, 1)
(10, 25, 3)
(25, 37, 3)
(11, 38, 4)
(12, 16, 2)
(27, 36, 2)
(13, 38, 4)
(14, 37, 3)
(15, 38, 4)
(31, 7, 1)
(32, 1, 2)
(33, 10, 3)
(34, 5, 4)


In [127]:
model_1.y.pprint()

y : Size=15, Index=y_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  -0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :  -0.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :  -0.0 :     1 : False : False : Binary
      8 :     0 :   0.0 :     1 : False : False : Binary
      9 :     0 :   0.0 :     1 : False : False : Binary
     10 :     0 :  -0.0 :     1 : False : False : Binary
     11 :     0 :   0.0 :     1 : False : False : Binary
     12 :     0 :  -0.0 :     1 : False : False : Binary
     13 :     0 :   0.0 :     1 : False : False : Binary
     14 :     0 :   0.0 :     1 : False : False : Binary
     15 :     0 :   0.0 :     1 : False : False : Binary


In [128]:
model_1.t.pprint()

t : Size=152, Index=t_index
    Key     : Lower : Value             : Upper : Fixed : Stale : Domain
     (1, 1) :  None :             383.0 :  None : False : False :  Reals
     (1, 2) :  None :             383.0 :  None : False : False :  Reals
     (1, 3) :  None :             383.0 :  None : False : False :  Reals
     (1, 4) :  None :             383.0 :  None : False : False :  Reals
     (2, 1) :  None :             149.0 :  None : False : False :  Reals
     (2, 2) :  None :              None :  None : False :  True :  Reals
     (2, 3) :  None :              None :  None : False :  True :  Reals
     (2, 4) :  None :              None :  None : False :  True :  Reals
     (3, 1) :  None :               5.0 :  None : False : False :  Reals
     (3, 2) :  None :               5.0 :  None : False : False :  Reals
     (3, 3) :  None :               5.0 :  None : False : False :  Reals
     (3, 4) :  None :              None :  None : False :  True :  Reals
     (4, 1) :  None :  

In [129]:
model_1.l.pprint()

l : Size=152, Index=l_index
    Key     : Lower : Value              : Upper : Fixed : Stale : Domain
     (1, 1) :     0 : 2368.0000000000246 :  None : False : False : NonNegativeReals
     (1, 2) :     0 :  2368.000000000019 :  None : False : False : NonNegativeReals
     (1, 3) :     0 :             2368.0 :  None : False : False : NonNegativeReals
     (1, 4) :     0 :             2368.0 :  None : False : False : NonNegativeReals
     (2, 1) :     0 :  9222.999999999998 :  None : False : False : NonNegativeReals
     (2, 2) :     0 :               None :  None : False :  True : NonNegativeReals
     (2, 3) :     0 :               None :  None : False :  True : NonNegativeReals
     (2, 4) :     0 :               None :  None : False :  True : NonNegativeReals
     (3, 1) :     0 :  2581.999999999959 :  None : False : False : NonNegativeReals
     (3, 2) :     0 : 3219.9999999999623 :  None : False : False : NonNegativeReals
     (3, 3) :     0 :             3171.0 :  None : False :

In [130]:
model_1.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 29590*x[1,1,1] + 58627*x[1,16,1] + 109747*x[1,2,1] + 74279*x[1,17,1] + 72303*x[1,3,1] + 49584*x[1,18,1] + 74279*x[1,4,1] + 83407*x[1,19,1] + 102849*x[1,5,1] + 38295*x[1,20,1] + 39851*x[1,6,1] + 102849*x[1,21,1] + 72160*x[1,7,1] + 74279*x[1,22,1] + 70835*x[1,8,1] + 96261*x[1,23,1] + 109747*x[1,10,1] + 111287*x[1,25,1] + 72303*x[1,11,1] + 71348*x[1,26,1] + 29590*x[1,12,1] + 111287*x[1,27,1] + 74261*x[1,13,1] + 98741*x[1,28,1] + 98741*x[1,15,1] + 80527*x[1,30,1] + 74921*x[1,31,1] + 55859*x[16,1,1] + 26822*x[16,16,1] + 85149*x[16,2,1] + 49681*x[16,17,1] + 47705*x[16,3,1] + 75516*x[16,18,1] + 49681*x[16,4,1] + 58809*x[16,19,1] + 128781*x[16,5,1] + 64227*x[16,20,1] + 65783*x[16,6,1] + 128781*x[16,21,1] + 47562*x[16,7,1] + 49681*x[16,22,1] + 46237*x[16,8,1] + 122193*x[16,23,1] + 85149*x[16,10,1] + 87026*x[16,25,1] + 47705*x[16,11,1] + 47836*x[16,26,1] + 55859*x[16,12,1] + 87026*x[16,

In [131]:
model_1.const_7.pprint()

const_7 : Size=38, Index=const_7_index, Active=True
    Key     : Lower : Body    : Upper   : Active
     (1, 1) :   0.0 :  l[1,1] : 16500.0 :   True
     (1, 2) :   0.0 :  l[1,2] :  5800.0 :   True
     (1, 3) :   0.0 :  l[1,3] :  3570.0 :   True
     (1, 4) :   0.0 :  l[1,4] :  3570.0 :   True
     (2, 1) :   0.0 :  l[2,1] : 16500.0 :   True
     (3, 1) :   0.0 :  l[3,1] : 16500.0 :   True
     (3, 2) :   0.0 :  l[3,2] :  5800.0 :   True
     (3, 3) :   0.0 :  l[3,3] :  3570.0 :   True
     (4, 1) :   0.0 :  l[4,1] : 16500.0 :   True
     (4, 2) :   0.0 :  l[4,2] :  5800.0 :   True
     (5, 1) :   0.0 :  l[5,1] : 16500.0 :   True
     (5, 2) :   0.0 :  l[5,2] :  5800.0 :   True
     (5, 3) :   0.0 :  l[5,3] :  3570.0 :   True
     (5, 4) :   0.0 :  l[5,4] :  3570.0 :   True
     (6, 1) :   0.0 :  l[6,1] : 16500.0 :   True
     (7, 1) :   0.0 :  l[7,1] : 16500.0 :   True
     (7, 2) :   0.0 :  l[7,2] :  5800.0 :   True
     (7, 3) :   0.0 :  l[7,3] :  3570.0 :   True
     (7, 4) :   0

In [132]:
"""from pyomo.util.infeasible import log_infeasible_constraints
import logging
log_infeasible_constraints(model_1, log_expression=True, log_variables=True)
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.INFO)"""

"from pyomo.util.infeasible import log_infeasible_constraints\nimport logging\nlog_infeasible_constraints(model_1, log_expression=True, log_variables=True)\nlogging.basicConfig(filename='example.log', encoding='utf-8', level=logging.INFO)"